In [ ]:
p_lakehouse_name = "<LAKEHOUSE_NAME>"
p_table_name = "<LAKEHOUSE_TABLE_NAME>"
p_lakehouse_sql_endpoint = "<LAKEHOUSE_SQL_ENDPOINT>"
p_azure_key_vault = "<AZURE_KEY_VAULT>"

# Key Vault secret keys for Service Principal
p_secret_client_id = "<SECRET_CLIENT_ID>"
p_secret_client_secret = "<SECRET_CLIENT_SECRET>"

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

kv_uri = f"https://{p_azure_key_vault}.vault.azure.net/"

credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=kv_uri, credential=credential)

client_id = secret_client.get_secret(p_secret_client_id).value
client_secret = secret_client.get_secret(p_secret_client_secret).value

In [ ]:
# Service Principal Authentication
import pyodbc

database = "dbo"
driver = '{ODBC Driver 18 for SQL Server}'
authentication = 'ActiveDirectoryServicePrincipal'

try:
    conn_str = f'DRIVER={driver};SERVER={p_lakehouse_sql_endpoint},1433;DATABASE={p_lakehouse_name};UID={client_id};PWD={client_secret};Authentication={authentication};Encrypt=Yes'

    with pyodbc.connect(conn_str, timeout=20) as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"SELECT TOP 10 * FROM {database}.{p_table_name}")
            row = cursor.fetchone()
            while row:
                print(row)
                row = cursor.fetchone()
except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
## Troubleshooting
!nc -zv f"{p_lakehouse_sql_endpoint}" 1433
!odbcinst -q -d -n
!pip list | grep -E "pyodbc|pyspark"